# Travel Map Code

## Get data from csv file

In [55]:
import os
import pandas as pd

location_list = os.path.join('/Users/ShawnHome/Desktop/', 'Travel_Log.csv')

locations = pd.read_csv(location_list, na_values="")  # CSV row and column data imported as DataFrame object

locations["State"] = " " + locations["State"]
#locations.fillna("", inplace=True)
locations["Full Name"] = locations["Address"].map(str) + " " + locations["City"].map(str) + locations["State"] + ", " + locations["Country"]

print(locations["Full Name"])

0                                         Rome, Italy
1                              San Marino, San Marino
2                                     Florence, Italy
3                                       Toledo, Spain
4                                       Madrid, Spain
5                                    Barcelona, Spain
6                                     Vienna, Austria
7                                   Salzburg, Austria
8                                  Innsbruck, Austria
9                                     Munich, Germany
10                                 Nuremburg, Germany
11                                    Dachau, Germany
12                                  Sydney, Australia
13                               Melbourne, Australia
14                  Ann Arbor Michigan, United States
15               East Lansing Michigan, United States
16                    Lansing Michigan, United States
17                                  Vancouver, Canada
18             Mount Shasta 

## Retrieve latitudes and longitudes
- (can take a while)

In [56]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

tqdm.pandas()  # only needed for text based progress bar

geolocator = Nominatim(user_agent="specify_your_app_name_here")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

locations['geocode'] = locations['Full Name'].progress_apply(geocode) # Just use "apply" if no progress bar
locations['coordinates'] = locations['geocode'].apply(lambda loc: tuple(loc.point) if loc else None)

print(locations)

100%|██████████| 101/101 [02:42<00:00,  1.73s/it]

    Address                       City        State               Country  \
0                                 Rome                              Italy   
1                           San Marino                         San Marino   
2                             Florence                              Italy   
3                               Toledo                              Spain   
4                               Madrid                              Spain   
5                            Barcelona                              Spain   
6                               Vienna                            Austria   
7                             Salzburg                            Austria   
8                            Innsbruck                            Austria   
9                               Munich                            Germany   
10                           Nuremburg                            Germany   
11                              Dachau                            Germany   

## Create Map and plot markers

In [81]:
import folium

my_map = folium.Map(location=[0.0, 0.0],zoom_start = 2)

#visited = folium.Icon(color='blue', icon='clipboard-check')
#home = folium.Icon(color='green', icon='home')
#favorite = folium.Icon(color='yellow', icon='star')
#wish = folium.Icon(color='red', icon='clipboard-list')

lived_group = folium.FeatureGroup(name='Where I\'ve Lived')
visit_group = folium.FeatureGroup(name='Where I\'ve Visited')
wish_group = folium.FeatureGroup(name='Where I Want To Visit')
other_group = folium.FeatureGroup(name='Other')

for index, city in locations.iterrows():
    #print(city['coordinates'])
    if city['coordinates'] is not None:
        #print(icon_type)
        if city['Lived']:
            icon_type = folium.Icon(color='darkgreen', icon_color='darkred', icon='home')
            group = lived_group
        elif city['Favorite']:
            icon_type = folium.Icon(color='darkblue', icon_color='orange', icon='star')
            group = visit_group
        elif city['Visited']:
            icon_type = folium.Icon(color='darkblue', icon_color='orange', icon='ok')
            group = visit_group
        elif city['Wish']:
            #print(city['coordinates'])
            icon_type = folium.Icon(color='lightblue', icon_color='cadetblue', icon='list-alt')
            group = wish_group
        else:
            icon_type = folium.Icon()
            group = other_group
        #print(icon_type)
        #icon_type = folium.Icon(color='green', icon='home')
        popup = folium.Popup(city["Full Name"], parse_html=True)
        folium.Marker([city['coordinates'][0], city['coordinates'][1]], 
                      popup=popup,
                      icon=icon_type,
                     ).add_to(group)

lived_group.add_to(my_map)
visit_group.add_to(my_map)
wish_group.add_to(my_map)
#other_group.add_to(my_map)
folium.LayerControl().add_to(my_map) 

my_map

## Color code visited states and countries

In [130]:
from numpy import nan

states = os.path.join('/Users/ShawnHome/Programming/github/folium/examples/data', 'us-states.json')
countries = os.path.join('/Users/ShawnHome/Programming/github/folium/examples/data', 'world-countries.json')
if not os.path.exists(states):
    print("Path of the file is Invalid")
    
colormap = pd.DataFrame()
colormap['States'] = locations.drop_duplicates(subset='State').get('State')
colormap.replace('', nan, inplace=True)
colormap.dropna(subset=['States'], inplace=True)
colormap.reset_index(drop=True, inplace=True)
colormap['Value'] = [1] * len(colormap['States'])
#print(colormap)

def style_function(feature):
    #state_visited = colormap['States']
    print(feature)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#blue' if colormap['States'].str.contains(feature['properties']['name']) else '#black'
    }

my_map = folium.Map(location=[48, -102],zoom_start = 3)

folium.GeoJson(
    open(states).read(),
    style_function=style_function
).add_to(my_map)


#my_map.choropleth(
#    geo_data=states,
#    name='States',
#    data=colormap,
#    columns=['States', 'Value'],
#    key_on='feature.properties.name',  #'feature.id',
#    fill_color='YlGn',
#    fill_opacity=0.7,
#    line_opacity=0.2
#)

#folium.LayerControl().add_to(my_map)

my_map

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama', 'style': {}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().